In [1]:
# 標準使用ライブラリー
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc
import os
import shutil


# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 追記
import json
import datetime
import math
plt.style.use('dark_background')

# debug
#%pdb on

import pixiedust #%pixie_debugger

# tfがエラーはかないため
#import tensorflow as tf
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)


Pixiedust database opened successfully


# 課題1

* Wikipediaの適当な英語のエントリをダウンロードする。

 * 選ぶのが面倒という方はAIのエントリでもどうぞ。
   * https://en.wikipedia.org/wiki/Artificial_intelligence

* BeautifulSoupで本文のテキストだけを取得する。

 * HTMLのソースを見て、どこが本文かを確認する。
 * あるいは、ネット検索をして、Wikipediaのエントリから本文だけを取得する方法を調べる。

* 以下の前処理をする。

 * 大文字は小文字にする。ただし固有名詞を除く。

 * ストップワードを除去する。

 * lemmatizationする。

* 各単語の出現回数を求め、表示する。

* lemmatizationした後の単語を、元の出現順序どおりに、半角スペースで区切ってつなぎ、長い一つの文字列にする。
 * joinメソッドを使えばよい。つまり、__`' '.join(`__ lemmatizeされた単語のリスト __`)`__ という感じ。

In [2]:
import requests
import re
from bs4 import BeautifulSoup
data = requests.get('https://en.wikipedia.org/wiki/Yoshi')
content = data.text

In [25]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    contents = soup.find('div', id="content")
#        soup.find('div', id="id_name")

    stripped_text = contents.get_text()
    # 下の正規表現の意味を説明してみよう。
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

In [26]:
clean_content = strip_html_tags(content)

In [27]:
print(clean_content)


Yoshi
From Wikipedia, the free encyclopedia
Jump to navigation
Jump to search
Nintendo character
This article is about the Nintendo character. For other uses, see Yoshi (disambiguation).
YoshiMario and Yoshi characterYoshi holding one of his eggs, as depicted in promotional artwork for Yoshi's Crafted WorldFirst appearanceSuper Mario World (1990)Last appearanceSuper Mario 3D All-Stars (2020)Created byShigefumi Hino[1]Voiced byKazumi Totaka (video games; 1997–present)Andrew Sabiston (Super Mario World)Frank Welker (Super Mario Bros.)In-universe informationFull nameT. Yoshisaur MunchakoopasSpeciesYoshiGenderMaleSignificant otherBirdo
Yoshi[a] is a fictional dinosaur[5] who appears in video games published by Nintendo.[6] Yoshi debuted in Super Mario World (1990) on the Super Nintendo Entertainment System[7] as Mario and Luigi's sidekick. Yoshi later starred in platform and puzzle games, including Super Mario World 2: Yoshi's Island, Yoshi's Story, Yoshi's Woolly World, and Yoshi's Craft

In [35]:
clean_content

'\nYoshi\nFrom Wikipedia, the free encyclopedia\nJump to navigation\nJump to search\nNintendo character\nThis article is about the Nintendo character. For other uses, see Yoshi (disambiguation).\nYoshiMario and Yoshi characterYoshi holding one of his eggs, as depicted in promotional artwork for Yoshi\'s Crafted WorldFirst appearanceSuper Mario World (1990)Last appearanceSuper Mario 3D All-Stars (2020)Created byShigefumi Hino[1]Voiced byKazumi Totaka (video games; 1997–present)Andrew Sabiston (Super Mario World)Frank Welker (Super Mario Bros.)In-universe informationFull nameT. Yoshisaur MunchakoopasSpeciesYoshiGenderMaleSignificant otherBirdo\nYoshi[a] is a fictional dinosaur[5] who appears in video games published by Nintendo.[6] Yoshi debuted in Super Mario World (1990) on the Super Nintendo Entertainment System[7] as Mario and Luigi\'s sidekick. Yoshi later starred in platform and puzzle games, including Super Mario World 2: Yoshi\'s Island, Yoshi\'s Story, Yoshi\'s Woolly World, and

In [48]:
clean_content = clean_content.replace('''
mw-parser-output cite.citation{font-style:inherit}.mw-parser-output .citation q{quotes:"\\"""\\"""\'""\'"}.mw-parser-output .id-lock-free a,.mw-parser-output .citation .cs1-lock-free a{background:linear-gradient(transparent,transparent),url("//upload.wikimedia.org/wikipedia/commons/6/65/Lock-green.svg")right 0.1em center/9px no-repeat}.mw-parser-output .id-lock-limited a,.mw-parser-output .id-lock-registration a,.mw-parser-output .citation .cs1-lock-limited a,.mw-parser-output .citation .cs1-lock-registration a{background:linear-gradient(transparent,transparent),url("//upload.wikimedia.org/wikipedia/commons/d/d6/Lock-gray-alt-2.svg")right 0.1em center/9px no-repeat}.mw-parser-output .id-lock-subscription a,.mw-parser-output .citation .cs1-lock-subscription a{background:linear-gradient(transparent,transparent),url("//upload.wikimedia.org/wikipedia/commons/a/aa/Lock-red-alt-2.svg")right 0.1em center/9px no-repeat}.mw-parser-output .cs1-subscription,.mw-parser-output .cs1-registration{color:#555}.mw-parser-output .cs1-subscription span,.mw-parser-output .cs1-registration span{border-bottom:1px dotted;cursor:help}.mw-parser-output .cs1-ws-icon a{background:linear-gradient(transparent,transparent),url("//upload.wikimedia.org/wikipedia/commons/4/4c/Wikisource-logo.svg")right 0.1em center/12px no-repeat}.mw-parser-output code.cs1-code{color:inherit;background:inherit;border:none;padding:inherit}.mw-parser-output .cs1-hidden-error{display:none;font-size:100%}.mw-parser-output .cs1-visible-error{font-size:100%}.mw-parser-output .cs1-maint{display:none;color:#33aa33;margin-left:0.3em}.mw-parser-output .cs1-subscription,.mw-parser-output .cs1-registration,.mw-parser-output .cs1-format{font-size:95%}.mw-parser-output .cs1-kern-left,.mw-parser-output .cs1-kern-wl-left{padding-left:0.2em}.mw-parser-output .cs1-kern-right,.mw-parser-output .cs1-kern-wl-right{padding-right:0.2em}.mw-parser-output .citation .mw-selflink{font-weight:inherit}\n
''',
'',1)

In [49]:
print(clean_content)


Yoshi
From Wikipedia, the free encyclopedia
Jump to navigation
Jump to search
Nintendo character
This article is about the Nintendo character. For other uses, see Yoshi (disambiguation).
YoshiMario and Yoshi characterYoshi holding one of his eggs, as depicted in promotional artwork for Yoshi's Crafted WorldFirst appearanceSuper Mario World (1990)Last appearanceSuper Mario 3D All-Stars (2020)Created byShigefumi Hino[1]Voiced byKazumi Totaka (video games; 1997–present)Andrew Sabiston (Super Mario World)Frank Welker (Super Mario Bros.)In-universe informationFull nameT. Yoshisaur MunchakoopasSpeciesYoshiGenderMaleSignificant otherBirdo
Yoshi[a] is a fictional dinosaur[5] who appears in video games published by Nintendo.[6] Yoshi debuted in Super Mario World (1990) on the Super Nintendo Entertainment System[7] as Mario and Luigi's sidekick. Yoshi later starred in platform and puzzle games, including Super Mario World 2: Yoshi's Island, Yoshi's Story, Yoshi's Woolly World, and Yoshi's Craft

In [6]:
!wget https://en.wikipedia.org/wiki/Yoshi

--2020-10-30 05:03:18--  https://en.wikipedia.org/wiki/Yoshi
Resolving en.wikipedia.org (en.wikipedia.org)... 103.102.166.224, 2001:df2:e500:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|103.102.166.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131358 (128K) [text/html]
Saving to: ‘Yoshi.1’

Yoshi.1             100%[===================>] 128.28K   561KB/s    in 0.2s    

2020-10-30 05:03:19 (561 KB/s) - ‘Yoshi.1’ saved [131358/131358]



In [7]:
%cd ./wikiextractor/

[Errno 2] No such file or directory: './wikiextractor/'
/home/input/course-nlp2020/report1


In [8]:
%ls

Yoshi  Yoshi.1  report1_20vr025e.ipynb


In [9]:
import sys
sys.path

['/home/input/course-nlp2020/report1',
 '/home/input/course-nlp2020/report1',
 '/opt/facets/facets_overview/python',
 '/opt/conda/lib/python37.zip',
 '/opt/conda/lib/python3.7',
 '/opt/conda/lib/python3.7/lib-dynload',
 '',
 '/root/.local/lib/python3.7/site-packages',
 '/opt/conda/lib/python3.7/site-packages',
 '/src/bq-helper',
 '/opt/conda/lib/python3.7/site-packages/IPython/extensions',
 '/root/.ipython']

In [10]:
sys.path.append(os.path.join(os.path.dirname('/home/input/course-nlp2020/wikiextractor/wikiextractor/')))

In [16]:
%cd /home/input/course-nlp2020/wikiextractor/wikiextractor/

/home/input/course-nlp2020/wikiextractor/wikiextractor


In [18]:
!python3 -m WikiExtractor.py ../../report1/Yoshi

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/runpy.py", line 183, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/opt/conda/lib/python3.7/runpy.py", line 109, in _get_module_details
    __import__(pkg_name)
  File "/home/input/course-nlp2020/wikiextractor/wikiextractor/WikiExtractor.py", line 62, in <module>
    from .extract import Extractor, ignoreTag
ImportError: attempted relative import with no known parent package


In [19]:
%pwd
!ls

'/home/input/course-nlp2020/wikiextractor/wikiextractor'

In [61]:
clre.sub("mw-parser-output.*ont-weight:inherit","",clean_content)

'\nYoshi\nFrom Wikipedia, the free encyclopedia\nJump to navigation\nJump to search\nNintendo character\nThis article is about the Nintendo character. For other uses, see Yoshi (disambiguation).\nYoshiMario and Yoshi characterYoshi holding one of his eggs, as depicted in promotional artwork for Yoshi\'s Crafted WorldFirst appearanceSuper Mario World (1990)Last appearanceSuper Mario 3D All-Stars (2020)Created byShigefumi Hino[1]Voiced byKazumi Totaka (video games; 1997–present)Andrew Sabiston (Super Mario World)Frank Welker (Super Mario Bros.)In-universe informationFull nameT. Yoshisaur MunchakoopasSpeciesYoshiGenderMaleSignificant otherBirdo\nYoshi[a] is a fictional dinosaur[5] who appears in video games published by Nintendo.[6] Yoshi debuted in Super Mario World (1990) on the Super Nintendo Entertainment System[7] as Mario and Luigi\'s sidekick. Yoshi later starred in platform and puzzle games, including Super Mario World 2: Yoshi\'s Island, Yoshi\'s Story, Yoshi\'s Woolly World, and